<a href="https://colab.research.google.com/github/ThanuMahee12/ayush-vision-idenification/blob/mobile-net/Algorithums/pretrained/MobileNetV3/MobileNetV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MobileNet v3

## Imports

### Collab Imports

In [1]:
from google.colab import files
from google.colab import drive

### Data Set Handle Imports

In [2]:
import zipfile

### Default Nessary Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

### Tensorflow Imports

In [4]:
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Dense,Conv2D, MaxPooling2D, Flatten,GlobalAveragePooling2D

## Variables

### Path variables

In [ ]:
dataset_zip_path="" # zip path in drive the dataset
extract_folder_path="" # when We extract the dataset

## Envirumentatl Setup

### Collab

In [ ]:
drive.mount('/content/drive') # drive activate for Use

In [ ]:
uploaded = files.upload() # for File uploaded

### Gentral

In [ ]:
with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder_path)                   #unzip Folder